## 2 Layer MLP w/InfiMNIST

10,000 epochs of RMSProp

In [1]:
import tensorflow as tf
import numpy as np
import input_data

In [2]:
%run infimnist.py

In [3]:
num_epochs = 10000
batch_size = 125

learning_rate = 0.001

In [4]:
# Network Parameters
n_hidden_1 = 1000 # 1st layer num features
n_hidden_2 = 500 # 2nd layer num features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

std_0 = 1.0/np.sqrt(n_input)
std_h1 = 1.0/np.sqrt(n_hidden_1)
std_h2 = 1.0/np.sqrt(n_hidden_2)

logfile = "2-layer-mlp-infimnist"

In [5]:
# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

xt = tf.placeholder("float", [None, n_input])
yt = tf.placeholder("float", [None, n_classes])

In [6]:
def multilayer_perceptron(_X, _weights, _biases):
    layer_1 = tf.nn.relu(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1'])) 
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, _weights['h2']), _biases['b2'])) 
    return tf.matmul(layer_2, weights['out']) + biases['out']

In [7]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=std_0)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=std_h1)),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes], stddev=std_h2))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1], stddev=0.1)),
    'b2': tf.Variable(tf.random_normal([n_hidden_2], stddev=0.01)),
    'out': tf.Variable(tf.random_normal([n_classes], stddev=0.001))
}

In [8]:
mlp = multilayer_perceptron(x, weights, biases )
mlp_test = multilayer_perceptron(xt, weights, biases )

In [9]:
cost =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(mlp, y)) 
gs = tf.get_variable("global_step",[],trainable=False,initializer=tf.constant_initializer(0))
lr = tf.constant(learning_rate) 

In [10]:
train_op = tf.contrib.layers.optimize_loss(cost, global_step=gs, learning_rate=lr,optimizer="RMSProp")

In [11]:
#with tf.name_scope("training accuracy"):
pred = tf.equal(tf.argmax(mlp, 1), tf.argmax(y, 1)) # Count correct predictions
train_acc_op = tf.reduce_mean(tf.cast(pred, "float"))  # Cast boolean to float to average
tf.scalar_summary("training accuracy", train_acc_op)

<tf.Tensor 'ScalarSummary_2:0' shape=() dtype=string>

In [12]:
test_pred = tf.equal(tf.argmax(mlp_test, 1), tf.argmax(yt, 1)) # Count correct predictions
test_acc_op = tf.reduce_mean(tf.cast(test_pred, "float"))  # Cast boolean to float to average
tf.scalar_summary("test 0 accuracy", test_acc_op)

<tf.Tensor 'ScalarSummary_3:0' shape=() dtype=string>

In [13]:
infiminst = InfiMNIST()

In [18]:
!rm -rf ./logs/{logfile}
!ls logs

### Original MNIST Data

In [19]:
infiminst.next_epoch()
mnist = input_data.read_data_sets(".", one_hot=True)
trX_0, trY_0 = mnist.train.images, mnist.train.labels
teX_0, teY_0 = mnist.test.images, mnist.test.labels

In [20]:
test_accuracies = []
train_accuracies = []
with tf.Session() as sess:
    # create a log writer. run 'tensorboard --logdir=./logs/{logfile}'
    writer = tf.train.SummaryWriter("./logs/{0}".format(logfile), sess.graph) # for 0.8
    merged = tf.merge_all_summaries()
 
    tf.initialize_all_variables().run()

    for epoch in range(num_epochs):
        infiminst.next_epoch()
        mnist = input_data.read_data_sets(".", one_hot=True)
    
        trX, trY = mnist.train.images, mnist.train.labels
        teX, teY = mnist.test.images, mnist.test.labels
        
        for start, end in zip(range(0, len(trX), batch_size), range(batch_size, len(trX), batch_size)):
            sess.run(train_op, feed_dict={x: trX[start:end], y: trY[start:end]})
                 
        summary, trn_acc, tst_acc = sess.run([merged, train_acc_op, test_acc_op], feed_dict={x: trX, y: trY, xt: teX_0, yt: teY_0})
        writer.add_summary(summary, epoch)  
        
        print(epoch, trn_acc, tst_acc)
        train_accuracies.append(trn_acc)
        test_accuracies.append(tst_acc)

        writer.flush()
        

(0, 0.94185454, 0.94980001)
(1, 0.95432729, 0.95069999)
(2, 0.97847271, 0.97640002)
(3, 0.98154545, 0.9774)
(4, 0.98556364, 0.98269999)
(5, 0.98334545, 0.98100001)
(6, 0.98569089, 0.98320001)
(7, 0.98067272, 0.97719997)
(8, 0.98710907, 0.98070002)
(9, 0.98974544, 0.9842)
(10, 0.98718184, 0.98089999)
(11, 0.98839998, 0.98119998)
(12, 0.99165457, 0.98519999)
(13, 0.99098182, 0.98500001)
(14, 0.98776364, 0.9806)
(15, 0.98810911, 0.98070002)
(16, 0.98830909, 0.98210001)
(17, 0.99249089, 0.98580003)
(18, 0.99201816, 0.9842)
(19, 0.98981816, 0.98479998)
(20, 0.99232727, 0.98430002)
(21, 0.98930907, 0.9824)
(22, 0.99189091, 0.98559999)
(23, 0.99352729, 0.98409998)
(24, 0.99241817, 0.9853)
(25, 0.99503636, 0.98769999)
(26, 0.99127275, 0.98470002)
(27, 0.99430907, 0.98589998)
(28, 0.99110907, 0.98540002)
(29, 0.98830909, 0.97780001)
(30, 0.99252725, 0.98629999)
(31, 0.99510908, 0.98720002)
(32, 0.99421817, 0.98589998)
(33, 0.99479997, 0.98470002)
(34, 0.99341816, 0.9853)
(35, 0.99414545, 0.9843

In [22]:
import matplotlib as plt
plt.hist(test_accuracies)

AttributeError: 'module' object has no attribute 'hist'